<h1>NBA Prediction 1.0<h1>

In [1]:
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import requests as rq
import time

In [2]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog

In [3]:
log = leaguegamelog.LeagueGameLog().get_normalized_dict()['LeagueGameLog']

Get the teams which will play next:

In [4]:
today = str(datetime.datetime.now()).split(' ')[0]
next_url = f"https://www.nba.com/games?date={today}"
next_html = rq.get(next_url)
soup = BeautifulSoup(next_html.text, 'html')

In [9]:
team_names = soup.find_all('span', {'class':'MatchupCardTeamName_teamName__3i23P'})
team_names = [team.text for team in team_names]

In [10]:
team_names

['Bucks',
 'Magic',
 'Wizards',
 'Heat',
 '76ers',
 'Raptors',
 'Lakers',
 'Rockets',
 'Knicks',
 'Timberwolves',
 'Cavaliers',
 'Pelicans',
 'Nuggets',
 'Warriors',
 'Thunder',
 'Kings']

Make a dataframe what we can use for the model training.

In [7]:
df1 = pd.DataFrame(log[::2])
df2 =  pd.DataFrame(log[1::2])
df2.columns = [f"{column}_2" for column in df2.columns]

all_games = df1.join(df2)

In [8]:
all_games_2 = all_games[['WL','TEAM_NAME','TEAM_NAME_2']]

In [9]:
all_games_2['WL'] = all_games_2['WL'].map(lambda x: 'H' if x == 'W' else 'A')

C:\Users\milan\AppData\Local\Temp/ipykernel_6528/1400147148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_games_2['WL'] = all_games_2['WL'].map(lambda x: 'H' if x == 'W' else 'A')


In [10]:
all_games_2.columns = ['label','home_team','away_team']

In [11]:
all_games_2['home_team'] = all_games_2['home_team'].map(lambda x: 'Los Angeles Clippers' if x == 'LA Clippers' else x)
all_games_2['away_team'] = all_games_2['away_team'].map(lambda x: 'Los Angeles Clippers' if x == 'LA Clippers' else x)

C:\Users\milan\AppData\Local\Temp/ipykernel_6528/3979758956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_games_2['home_team'] = all_games_2['home_team'].map(lambda x: 'Los Angeles Clippers' if x == 'LA Clippers' else x)
C:\Users\milan\AppData\Local\Temp/ipykernel_6528/3979758956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_games_2['away_team'] = all_games_2['away_team'].map(lambda x: 'Los Angeles Clippers' if x == 'LA Clippers' else x)


In [48]:
def get_team_wpct(team):
    id = teams.find_teams_by_full_name(team)[0]['id']
    print(team)
    team_log = pd.DataFrame(teamgamelog.TeamGameLog(team_id=[id]).get_normalized_dict()["TeamGameLog"])
    time.sleep(.600)
    print(team_log.values[0])
    # return team_log.loc[(datetime.datetime.strptime(team_log['GAME_DATE'], "%b %d, %Y")) <= datetime.now(),['W_PCT']][0]

In [49]:
get_team_wpct('Chicago Bulls')

Chicago Bulls
[1610612741 '0022100499' 'DEC 26, 2021' 'CHI vs. IND' 'W' 20 10 0.667 240
 43 76 0.566 11 23 0.478 16 22 0.727 2 38 40 17 5 6 15 20 113]


In [22]:
all_games_2['home_wpct'] = all_games_2['home_team'].map(get_team_wpct)

Golden State Warriors


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
all_games_2['away_wpct'] = all_games_2['away_team'].map(get_team_wpct)